In [1]:
import os

In [2]:
%pwd  # this tell us which path we are currently working , so based on the below output path we are working under the research file 

'c:\\ProjectML\\ProjectML_with_MLFlow\\research'

In [3]:
os.chdir("../")  #  but i would like to work with main ProjectML_with_MLFlow file , so for getting i step back in path inorder to enter the main project file i used this command os.chdir("../") 

In [4]:
%pwd

'c:\\ProjectML\\ProjectML_with_MLFlow'

#### updating the entity with class 

In [5]:
# This is called the entity 
from dataclasses import dataclass # here i imported the dataclass from the dataclasses
from pathlib import Path  # here i imported path from pathlib

@dataclass(frozen=True)
class DataTransformationConfig:
    root_dir: Path  # these are variables which are present inside the config.yaml file data_transformation code part and here iam mentioning inside the entity of the class
    data_path: Path

#### updating the src manager configuration

In [6]:
from PROJECTML.constants import *
from PROJECTML.utils.common import read_yaml, create_directories

In [7]:
# this is same part of the code in every step 
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])


  # # only this part get changes in every step, only defining the get_data_transformation_config get changes according to which step we are performing like 01_data_ingestion,02_data_validation  
    def get_data_transformation_config(self) -> DataTransformationConfig:
        config = self.config.data_transformation

        create_directories([config.root_dir])

        data_transformation_config = DataTransformationConfig(
            root_dir=config.root_dir,  # here iam returning these 2 varaibles by using this code 
            data_path=config.data_path,
        )

        return data_transformation_config

#### updating the components 

In [9]:
import os
from PROJECTML import logger
from sklearn.model_selection import train_test_split
import pandas as pd

In [10]:
# here i defined the component of DataTransformationConfig below
class DataTransformation:
    def __init__(self, config: DataTransformationConfig):
        self.config = config

    
    ## Note: You can add different data transformation techniques such as Scaler, PCA and all
    #You can perform all kinds of EDA in ML cycle here before passing this data to the model

    # I am only adding train_test_spliting cz this data is already cleaned up


# here i have defined the tarin_test_split below for performing the train_test_split
    def train_test_spliting(self):
        data = pd.read_csv(self.config.data_path) # this line helps us to read the data

        # Split the data into training and test sets. (0.75, 0.25) split.
        train, test = train_test_split(data) # this line splits the data into train_test_split

        train.to_csv(os.path.join(self.config.root_dir, "train.csv"),index = False) # here it saves the train and test data in csv format inisde the artifacts-> transformation folder
        test.to_csv(os.path.join(self.config.root_dir, "test.csv"),index = False)

        logger.info("Splited data into training and test sets")
        logger.info(train.shape) # this logs the information about that how many training and testing samples i have 
        logger.info(test.shape)

        print(train.shape)
        print(test.shape)
        

### updating the Pipeline

In [12]:
try:
    config = ConfigurationManager() # here iam initlizing my ConfigurationManager
    data_transformation_config = config.get_data_transformation_config() # and here iam getting my get_data_transformation_config()
    data_transformation = DataTransformation(config=data_transformation_config) # here iam passing my data_transformation_config it means iam calling this data_transformation_config
    data_transformation.train_test_spliting() # here performing the train_test_split()
except Exception as e: # this part of code will raise error if anything goes wrong
    raise e

[2023-12-11 14:11:10,271: INFO: common: yaml file: config\config.yaml loaded successfully]
[2023-12-11 14:11:10,271: INFO: common: yaml file: params.yaml loaded successfully]
[2023-12-11 14:11:10,271: INFO: common: yaml file: schema.yaml loaded successfully]
[2023-12-11 14:11:10,279: INFO: common: created directory at: artifacts]
[2023-12-11 14:11:10,281: INFO: common: created directory at: artifacts/data_transformation]


[2023-12-11 14:11:10,355: INFO: 3339085162: Splited data into training and test sets]
[2023-12-11 14:11:10,355: INFO: 3339085162: (1199, 12)]
[2023-12-11 14:11:10,362: INFO: 3339085162: (400, 12)]
(1199, 12)
(400, 12)
